# Optimizer
## Proyecto final -- Optimización y Simulación
### Alejandro de Haro, Jose María Benítez, Daniel Escobosa, Pablo Berástegui

In [1]:
import pandas as pd
import pyomo.environ as pe
import pyomo.opt as po
import numpy as np

In [2]:
df = pd.read_csv('../data/players_fifa18_sample.csv')

In [3]:
chemistry = pd.read_csv('../data/sample_chemistry_edges_fifa18.csv')

In [4]:
chemistry.head()

,player_id_i,player_id_j,chem
0,188528,223862,0
1,201818,223862,0
2,223862,236001,0
3,189691,223862,0
4,191545,223862,0


In [6]:
model = pe.ConcreteModel()

### Sets

In [7]:
# usar player_id como conjunto para que coincida con el CSV de química
model.player = pe.Set(initialize=df['player_id'].astype(int).tolist())
model.club = pe.Set(initialize=df['club_team_id'].unique().tolist())
model.position = pe.Set(initialize=df['position'].unique().tolist())
model.nationality = pe.Set(initialize=df['nationality_id'].unique().tolist())
model.league = pe.Set(initialize=df['league_id'].unique().tolist())

### Parameters

In [9]:
# overall como dict por player_id
overall_dict = df.set_index('player_id')['overall'].astype(float).to_dict()
model.overall = pe.Param(model.player, initialize=overall_dict, within=pe.NonNegativeReals)

In [11]:
# construir dict de química (asegurar int)
chemistry_dict = { (int(r.player_id_i), int(r.player_id_j)): float(r.chem)
                   for _, r in chemistry.iterrows() }

# si quieres que la química sea simétrica (chem[i,j] == chem[j,i]) descomenta:
chem_sym = {}
for (i,j), w in chemistry_dict.items():
    chem_sym[(i,j)] = w
    chem_sym[(j,i)] = w
chemistry_dict = chem_sym

model.chemistry = pe.Param(model.player, model.player,
                           initialize=chemistry_dict,
                           default=0.0,
                           within=pe.NonNegativeReals)

Conexiones de la formación 4-3-3

In [7]:
                      #  P LB LCB RCB RB LCM CM RCM LW ST RW
conections = np.array([ [0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0],
                        [0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0],
                        [1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0],
                        [1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0],
                        [0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0],
                        [0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0],
                        [0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 0],
                        [0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1],
                        [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0],
                        [0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1],
                        [0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0]
])

Comprobación de simetría

In [22]:
conections.T == conections

array([[ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  True,  True,
         True,  True],
       [ True,  True,  True,  True,  True,  True,  True,  Tr